In [1]:
import numpy as np
from LightTools import LTAPIx
import time
import matplotlib.pyplot as plt
PID = 26652
loc = LTAPIx.LTLocator()
lt = loc.GetLTAPIFromPID(PID)
lt.Message('Script LT connection established')

0

In [8]:
timestr = time.strftime("%Y%m%d-%H-%M-%S")
lt.Message(timestr)
lt.Cmd("\O" + lt.Str('PROPERTY_MANAGER[Optical Properties Manager]'))
lt.Cmd("AddNew=""")
lt.Cmd("\Q")
lt.DbGet('PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[@Last]', 'Name')
lt.DbSet('PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[@Last]','Name',f'_{timestr}')

0

In [44]:
import re

def check_tabulated(s):
    # Use regex to find a number inside square brackets
    match = re.search(r'([^\[]+)\[(\d+)\]', s)
    if match:
        text_before_bracket = match.group(1)  # Extract the text before the bracket
        number_in_bracket = int(match.group(2))  # Extract the number inside the bracket as an integer
        return text_before_bracket, number_in_bracket
    return s, None  # Return None if the pattern isn't found

def set_zone_property(lt, name, paras):
    lt.Cmd("\O" + lt.Str('PROPERTY_MANAGER[Optical Properties Manager]'))
    lt.Cmd("AddNew=""")
    lt.Cmd("\Q")
    lt.DbSet('PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[@Last]','Name', f'{name}')
    lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}]', 'Simple Type', 'Linear Grating')    
    lt.SetOption("DbUpdate", 0)
    for f in paras.keys():
        for k_str in paras[f].keys():
            # lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k])
            k, ind = check_tabulated(k_str)
            if  ind is not None:
                # print(k, ind, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, None, ind))
                lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k_str], ind)
            else:                
                # print(k, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k))
                lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k_str])
    lt.SetOption("DbUpdate", 1)

def array_to_dict(input_array):
    output_dict = {}
    for i, (x, y) in enumerate(input_array, start=1):
        output_dict[f"Vertex_X_At[{i}]"] = x
        output_dict[f"Vertex_Y_At[{i}]"] = y
    return output_dict

def set_zone(lt, surface_key, name, shape, paras):
    lt.Cmd("\O" + lt.Str(surface_key))
    lt.Cmd(lt.Str("Add Property Zone"))
    lt.Cmd("\Q")
    lt.DbSet(f'{surface_key}.ZONE[@Last]','Name', f'{name}')
    lt.DbSet(f'{surface_key}.ZONE[{name}].RECT_ZONE_EXTENT[RectangularZone]', 'Element Shape', shape) 
    shape_keys = {'Circle':'CIRC_ZONE_EXTENT[CircularZone]', 'Polygonal':'POLYGON_ZONE_EXTENT[PolygonalZone]'}
    lt.SetOption("DbUpdate", 0)
    for k_str in paras.keys():        
        k, ind = check_tabulated(k_str)
        if  ind is not None:
            # print(k, ind, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, None, ind))
            lt.DbSet(f'{surface_key}.ZONE[{name}].{shape_keys[shape]}', k, paras[k_str], ind)
        else:                
            # print(k, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k))
            lt.DbSet(f'{surface_key}.ZONE[{name}].{shape_keys[shape]}', k, paras[k_str])    
    lt.SetOption("DbUpdate", 1)

In [17]:
paras_1 = {'RULED_DOE_DIRECTION_ZONE[RuledGrating]':{'Grating_Period':0.0005, 'Blaze_Type': 'Manual', 'Grating_Vector_X':0, 'Grating_Vector_Y':1, 'Grating_Vector_Z':0},
           'DIFFRACTION_EFFICIENCY_ZONE[diffEffTable]':{'Minimum_Order':0, 'Maximum_Order':1, 'EnabledTransmittedOrderAt[1]':0, 'ReflectedEfficiencyAt[1]':0.9, 'EnabledTransmittedOrderAt[2]':0, 
                                                        'ReflectedEfficiencyAt[2]':0.1, 'TransmittedEfficiencyAt[1]':0, 'TransmittedEfficiencyAt[2]':0}
           }

paras_2 = {'RULED_DOE_DIRECTION_ZONE[RuledGrating]':{'Grating_Period':0.0005, 'Blaze_Type': 'Manual', 'Grating_Vector_X':1, 'Grating_Vector_Y':0, 'Grating_Vector_Z':0},
           'DIFFRACTION_EFFICIENCY_ZONE[diffEffTable]':{'Minimum_Order':0, 'Maximum_Order':1, 'EnabledTransmittedOrderAt[1]':0, 'ReflectedEfficiencyAt[1]':0.8, 'EnabledTransmittedOrderAt[2]':0, 
                                                        'ReflectedEfficiencyAt[2]':0.2, 'TransmittedEfficiencyAt[1]':0, 'TransmittedEfficiencyAt[2]':0}
           }

In [45]:
name_list = ['test_1', 'test_2']
para_list = [paras_1, paras_2]
input = dict(zip(name_list, para_list))
for n in input.keys():
    set_zone_property(lt, n, input[n])

In [11]:
name_list = ['test_1', 'test_2']
type_list = ['Holographic Grating', 'Linear Grating']
input = dict(zip(name_list, type_list))

In [13]:
for k in input:
    set_zone_property(lt, k, input[k], None)

In [51]:
# Polygon area parameter input examples
paras_1 = {'X_Offset':0, 'Y_Offset':0, 'Theta_Rotation':0, 'Vertex_X_At[1]':0.8, 'Vertex_Y_At[1]':0.8, 'Vertex_X_At[2]':-0.8, 'Vertex_Y_At[2]':0.8, 'Vertex_X_At[3]':-0.8, 'Vertex_Y_At[3]':-0.8,
           'Vertex_X_At[4]':0.8, 'Vertex_Y_At[4]':-0.8
           }

paras_2 = {'X_Offset':2, 'Y_Offset':2, 'Theta_Rotation':30, 'Vertex_X_At[1]':0.8, 'Vertex_Y_At[1]':0.8, 'Vertex_X_At[2]':-0.8, 'Vertex_Y_At[2]':0.8, 'Vertex_X_At[3]':-0.8, 'Vertex_Y_At[3]':-0.8,
           'Vertex_X_At[4]':0.8, 'Vertex_Y_At[4]':-0.8
           }

# Circle area parameter input examples:
paras_3 = {'X_Offset':2, 'Y_Offset':2, 'Theta_Rotation':30, 'Radius':0.8}         

In [48]:
surface_key = 'LENS_MANAGER[1].COMPONENTS[Components].SOLID[Cylinder_1].CYLINDER_PRIMITIVE[CylinderPrimitive_1].PLANAR_SURFACE[FrontSurface]'

In [52]:
timestr = time.strftime("%Y%m%d-%H-%M-%S")
set_zone(lt, surface_key, timestr, 'Polygonal', paras_1)
timestr = time.strftime("%Y%m%d-%H-%M-%S")
set_zone(lt, surface_key, timestr, 'Polygonal', paras_2)

In [29]:
lt.DbGet('COMPONENTS[Components].SOLID[Cylinder_1].CYLINDER_PRIMITIVE[CylinderPrimitive_1].PLANAR_SURFACE[FrontSurface].ZONE[20240923-17-17-02].CIRC_ZONE_EXTENT[CircularZone]', 'Element Shape')

('Circle', 0)

In [18]:
class Zone:
    def __init__(self, name, shape_type, shape_paras, opt_prop, opt_paras) -> None:
        self.name = name
        self.shape_type = shape_type        
        self.shape_paras = shape_paras
        self.opt_prop = opt_prop
        self.opt_paras = opt_paras
        if shape_type == 'Polygonal':
            self.shape_paras_LT = {**{'X_Offset':0, 'Y_Offset':0, 'Theta_Rotation':0}, **self.poly_pts_to_dict(shape_paras)}
        elif shape_type == 'Circle':
            self.shape_paras_LT = {'X_Offset':shape_paras[0], 'Y_Offset':shape_paras[1], 'Theta_Rotation':0, 'Radius':shape_paras[2]}
        else:
            raise NameError('Only Polygonal and Circle are allowed')
        return
    
    def check_tabulated(self, s):
        # Use regex to find a number inside square brackets
        match = re.search(r'([^\[]+)\[(\d+)\]', s)
        if match:
            text_before_bracket = match.group(1)  # Extract the text before the bracket
            number_in_bracket = int(match.group(2))  # Extract the number inside the bracket as an integer
            return text_before_bracket, number_in_bracket
        return s, None  # Return None if the pattern isn't found

    def set_zone_property(self, lt, name, paras):
        lt.Cmd("\O" + lt.Str('PROPERTY_MANAGER[Optical Properties Manager]'))
        lt.Cmd("AddNew=""")
        lt.Cmd("\Q")
        lt.DbSet('PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[@Last]','Name', f'{name}')
        lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}]', 'Simple Type', 'Linear Grating')    
        lt.SetOption("DbUpdate", 0)
        for f in paras.keys():
            for k_str in paras[f].keys():
                # lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k])
                k, ind = self.check_tabulated(k_str)
                if  ind is not None:
                    # print(k, ind, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, None, ind))
                    lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k_str], ind)
                else:                
                    # print(k, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k))
                    lt.DbSet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, paras[f][k_str])
        lt.SetOption("DbUpdate", 1)

    def poly_pts_to_dict(self, input_array):
        output_dict = {}
        for i, (x, y) in enumerate(input_array, start=1):
            output_dict[f"Vertex_X_At[{i}]"] = x
            output_dict[f"Vertex_Y_At[{i}]"] = y
        return output_dict

    def set_zone(self, lt, surface_key, name, shape, paras):
        lt.Cmd("\O" + lt.Str(surface_key))
        lt.Cmd(lt.Str("Add Property Zone"))
        lt.Cmd("\Q")
        lt.DbSet(f'{surface_key}.ZONE[@Last]','Name', f'{name}')
        lt.DbSet(f'{surface_key}.ZONE[{name}].RECT_ZONE_EXTENT[RectangularZone]', 'Element Shape', shape) 
        shape_keys = {'Circle':'CIRC_ZONE_EXTENT[CircularZone]', 'Polygonal':'POLYGON_ZONE_EXTENT[PolygonalZone]'}
        lt.SetOption("DbUpdate", 0)
        for k_str in paras.keys():        
            k, ind = self.check_tabulated(k_str)
            if  ind is not None:
                # print(k, ind, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k, None, ind))
                lt.DbSet(f'{surface_key}.ZONE[{name}].{shape_keys[shape]}', k, paras[k_str], ind)
            else:                
                # print(k, lt.DbGet(f'PROPERTY_MANAGER[Optical Properties Manager].PROPERTY[{name}].{f}', k))
                lt.DbSet(f'{surface_key}.ZONE[{name}].{shape_keys[shape]}', k, paras[k_str])    
        lt.SetOption("DbUpdate", 1)
        
    def to_LT(self, lt, surface_key):
        self.set_zone(lt, surface_key, self.name, self.shape_type, self.shape_paras_LT)
        # self.set_zone_property(lt, self.name, self.opt_paras)
        return
    


In [12]:
import numpy as np
from LightTools import LTAPIx
import time
import matplotlib.pyplot as plt
import re
PID = 14324
loc = LTAPIx.LTLocator()
lt = loc.GetLTAPIFromPID(PID)
lt.Message('Script LT connection established')


0

In [22]:

surface_key = 'LENS_MANAGER[1].COMPONENTS[Components].SOLID[Cylinder_3].CYLINDER_PRIMITIVE[CylinderPrimitive_1].PLANAR_SURFACE[FrontSurface]'


epe_shape_pts = np.loadtxt('EPE.txt')
oc_shape_pts = np.loadtxt('oc.txt')


ic = Zone('IC', 'Circle', (-41.89725171, 29.00955127, 1.5), None, None)
epe = Zone('EPE', 'Polygonal', epe_shape_pts, None, None)
oc = Zone('OC', 'Polygonal', oc_shape_pts, None, None)


In [23]:

ic.to_LT(lt, surface_key)
epe.to_LT(lt, surface_key)
oc.to_LT(lt, surface_key)
